In [4]:
import networkx as nx
from sympy import Matrix, pprint, floor
from linkages import *
import openmesh as om
import numpy as np

def graph_to_matrix(G):
    M = Matrix()
    for edge in G.edges:
        row = []
        d = edge[0] - edge[1]

        for vertex in G.nodes:
            if vertex == edge[0]  : row.extend(d)
            elif vertex == edge[1]: row.extend(-d)
            else:                   row.extend([0 for _ in range(DIM)])
        M=Matrix([M,row])
    return M

def set_pinning(pins, M: Matrix):
    N = M.nullspace()
    if type(pins) is int: pins = [pins]

    for p in pins:
        for vector in N:
            for i in range(DIM):
                if vector[DIM*(p-1) + i] != 0:
                    vector.fill(0)
                    break
    return N

# helper function to convert nullspace to a list of motions 
def getMotions(N):
    if type(N) is Matrix: N = N.nullspace()
    return [[format(float(val), '.15f') + "*v" + str(floor(i/DIM)+1) + ("xyzwrüdiger"[i%DIM]) for i,val in enumerate(vector) if val != 0] for vector in N]

#just a cute function to convert detected motions into a human readible string
def motions_to_string(motions):
    string = ""
    for v in motions:
        if len(v) !=0:
            for val in v:
                if val == v[0] and len(v)>1: word = " depends on "
                elif val != v[len(v)-1]    : word = ", and "
                elif len(v) == 1           : word = " is free\n"
                else                       : word ="\n"
                string += val + word
    return string

def model_to_graph(mesh: om.PolyMesh) -> nx.Graph:
    mesh.update_normals() #lol
    graph = nx.Graph()

    points = mesh.points()

    rng = np.random.default_rng()

    wiggled_points = [Point(point + rng.random(DIM) * 1e-5) for point in points]

    for edge in mesh.edge_vertex_indices():
        graph.add_edge(wiggled_points[edge[0]], wiggled_points[edge[1]])

    for face in mesh.faces():
        neighbouring_faces = mesh.ff(face)
        for neighbour in neighbouring_faces:
            if np.allclose(mesh.normal(face), mesh.normal(neighbour)):
                vertices_face = mesh.fv(face)
                vertices_neighbour = mesh.fv(neighbour)
                first = [vertex for vertex in vertices_face if vertex not in vertices_neighbour]
                second = [vertex for vertex in vertices_neighbour if vertex not in vertices_face]
                first_point = wiggled_points[first[0].idx()]
                second_point = wiggled_points[second[0].idx()]
                if not graph.has_edge(first_point, second_point):
                    graph.add_edge(first_point, second_point)

    return graph    


graph = model_to_graph(om.read_polymesh("models/paperplane.stl"))

def check_rigidity(M): return M.rank() == M.cols- (DIM+1) * DIM/2

A = graph_to_matrix(graph)
pprint(A)
print ("the linkage is infinitesimally rigid!" if check_rigidity(A) else "the linkage is infinitesimally flexible")
# A = set_pinning([1],A)

print(motions_to_string((getMotions(A))))



⎡  20321991743703    165817980458641  -50552721672113    -20321991743703    -1
⎢  ──────────────    ───────────────  ────────────────   ────────────────   ──
⎢  25000000000000     50000000000000   25000000000000     25000000000000      
⎢                                                                             
⎢-2908303054901009   457352066279881  -29123804902961                         
⎢──────────────────  ───────────────  ────────────────          0             
⎢ 1000000000000000   100000000000000   20000000000000                         
⎢                                                                             
⎢                                                       -3721182724649129    1
⎢        0                  0                0          ──────────────────   ─
⎢                                                        1000000000000000    1
⎢                                                                             
⎢                                                   